# Proyecto práctico: árbol de decisión y random forest con scikit-learn

In [1]:
#Importamos las librerias principales
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
from pprint import pprint as pp

Utilizaremos el **Car Evaluation Data Set** de Kaggle: https://www.kaggle.com/datasets/elikplim/car-evaluation-data-set

In [2]:

path="/home/dan/PLATZI/data/Fundamentals/randomforestsalgorithms/data/external"
colum_names=["buying price","maintinance price","number of doors","persons capacity","luggage boot","safty stimated","target"]

#Cargamos dataset a utilizar
df=pd.read_csv(path+"/car_evaluation.csv")
df.columns=colum_names
categorical= df.select_dtypes(include=['object']).columns
uniqueValsSumary={
    col:{'unique count':df[col].nunique(),'unique values':df[col].unique().tolist()}
    for col in categorical
    
}
uniqueValsSumary=pd.DataFrame.from_dict(uniqueValsSumary,orient='index')
uniqueValsSumary


,unique count,unique values
buying price,4,"[vhigh, high, med, low]"
maintinance price,4,"[vhigh, high, med, low]"
number of doors,4,"[2, 3, 4, 5more]"
persons capacity,3,"[2, 4, more]"
luggage boot,3,"[small, med, big]"
safty stimated,3,"[med, high, low]"
target,4,"[unacc, acc, vgood, good]"


it apear all columns are categorical  by default even the ones that use numbers, a proper encoding will have to be in order

## Análisis exploratorio de datos

In [3]:
#Visualizacion del dataframe
df.head(3)

,buying price,maintinance price,number of doors,persons capacity,luggage boot,safty stimated,target
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc


In [4]:
#Analizamos el shape del objeto
print("size of the data frame")
print(df.size)
print("="*64)
print("frecuenzy of each category")
for col in df.columns:
    print(f"Value counts for: {col}")
    print(df[col].value_counts())
    print("\n")


size of the data frame
12089
frecuenzy of each category
Value counts for: buying price
buying price
high     432
med      432
low      432
vhigh    431
Name: count, dtype: int64


Value counts for: maintinance price
maintinance price
high     432
med      432
low      432
vhigh    431
Name: count, dtype: int64


Value counts for: number of doors
number of doors
3        432
4        432
5more    432
2        431
Name: count, dtype: int64


Value counts for: persons capacity
persons capacity
4       576
more    576
2       575
Name: count, dtype: int64


Value counts for: luggage boot
luggage boot
med      576
big      576
small    575
Name: count, dtype: int64


Value counts for: safty stimated
safty stimated
med     576
high    576
low     575
Name: count, dtype: int64


Value counts for: target
target
unacc    1209
acc       384
good       69
vgood      65
Name: count, dtype: int64




the target  classes are unbalance ofcourse here the numbers
- unacc    1209
- acc       384
- good       69
- vgood      65
all the other columns are more less at equilibrium so the distributions are uniform, there are no missing values .

In [5]:
#we will separate and train a class tree not having on acount the umbalance
X=df.drop(["target"],axis=1)
Y=df["target"]
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3, random_state=37)
verification={"xtrain":X_train.shape,"xtest":X_test.shape,"ytrain":Y_train.shape,"Ytest":Y_test.shape}
import json
# esto esta echo para terminal no para el text output de los jupyter
#pp(json.dumps(verification,indent=2))


In [7]:
#encoding using third party encoder because:
    #random forest dont bennefit from higth sparse matrix
    # one hot doesnt suport higth cardinality well
    # the encodings are target inform
    # good integration with sklearn pipelines
    #what is target leakage in this context?
#import category_encoders as ce and is not functionning because outdated
from sklearn.preprocessing import OrdinalEncoder 
colum_names=["buying price","maintinance price","number of doors","persons capacity","luggage boot","safty stimated"]
encoder=OrdinalEncoder()
X_train=encoder.fit_transform(X_train)
X_test=encoder.transform(X_test)
from sklearn.tree  import DecisionTreeClassifier
tree=DecisionTreeClassifier(max_depth=2,random_state=37)
tree.fit(X_train,Y_train)
train_predictions=tree.predict(X_train)
predictions=tree.predict(X_test)

In [8]:
#to evaluate a model any model its important to be clear on 
#whats a confussion matrix
#sensitibity or recall: all true how many picked; acuracy : the ones clasified how many true
#pressicion: from  of all that where say true  how many hhere positive true
# espesificity : proportions of true negatives
#F1 score integrates presicion and sensibility 
from sklearn.metrics import accuracy_score
#cheking for over fitting
train_accuracy=accuracy_score(Y_train,train_predictions)
accuracy=accuracy_score(Y_test,predictions)
print(f"train accuracy:{np.round(train_accuracy,2)}, and test accuracy: {np.round(accuracy,2)}")


train accuracy:0.77, and test accuracy: 0.76


as you see the accuracy in both scenarios is simillar so  there is no over or under fitting, never the lees the accuracy should be  o between 90 and 80 porcent to have a good model., lets traina model with a more balance data set to see what happens


In [ ]:
print(Y_train.dtype,X_train.dtype)




In [15]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=37)
Y_train.astype('category')
print(Y_train.dtype)
# ros dont work easyly with no numeric features.
#X_resampled, y_resampled = ros.fit_resample(X_train, Y_train)
#stratified sampling  ensures the train test split respects the original dataset ratio
X_balanced, X_test, Y_balanced,Y_test=train_test_split(X,Y,test_size=0.3, random_state=37,stratify= Y)
balancedTree=DecisionTreeClassifier(max_depth=2,random_state=37,class_weight='balanced')
#balancedTree.fit(X_resampled,y_resampled)
#resampled_predictions=balancedTree.predict(X_resampled)
balancedTree.fit(X_balanced,Y_balanced)
train_predictions=balancedTree.predict(X_train)
predictions= balancedTree.predict(X_test)
#accuracy mesures
#resampled_accuracy=accuracy_score(y_resampled,resampled_predictions)
train_accuracy=accuracy_score(Y_train,train_predictions)
accuracy=accuracy_score(Y_test,predictions)


object


ValueError: could not convert string to float: 'high'

In [ ]:
import sklearn
import imblearn
print("scikit-learn:", sklearn.__version__)
print("imbalanced-learn:", imblearn.__version__)
#Verificamos valores missings


ImportError: cannot import name '_deprecate_Xt_in_inverse_transform' from 'sklearn.utils.deprecation' (/home/dan/anaconda3/envs/desicionTrees/lib/python3.11/site-packages/sklearn/utils/deprecation.py)

## Procesamiento de datos

In [ ]:
#Separamos en X e y


In [ ]:
#Importamos las librerias necesarias para la creacion del modelo


#30% para test y 70% para train


In [ ]:
#Veamos que obtuvimos


In [ ]:
#Veamos que tenemos. Por ejemplo, en X_train


## Entrenamiento de modelo de clasificación con árbol de decisión

In [ ]:
#Importante: todos nuestros tipos de datos son object, realizamos una transformacion


In [ ]:
#Verificamos la transformacion


In [ ]:
#Importar árbol de decisión

#Creacion del modelo


In [ ]:
#Entrenamiento


In [ ]:
#Calculo de las predicciones en Train y Test


## Evaluación de modelo de clasificación con árbol de decisión

In [ ]:
#Calculo de metricas 


#Calculo el accuracy en Train


#Calculo el accuracy en Test


In [ ]:
#Verificamos el feature importances


## Entrenamiento de modelo de clasificación con random forest

In [ ]:
#Importar random forest


In [ ]:
#Calculo de las predicciones en Train y Test


## Evaluación de modelo de clasificación con random forest

In [ ]:
#Calculo de metricas 


#Calculo el accuracy en Train


#Calculo el accuracy en Test


#Importante: podriamos reducir el numero de estimadores para disminuir el sobreajuste del modelo.

In [ ]:
# Visualizacion de las feature importantes


In [ ]:
#Grafico de barras


In [ ]:
# Matriz de confusion del RF


In [ ]:
#RF
